https://huggingface.co/akahana/indonesia-sentiment-roberta

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="akahana/indonesia-sentiment-roberta")

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("akahana/indonesia-sentiment-roberta")
model = AutoModelForSequenceClassification.from_pretrained("akahana/indonesia-sentiment-roberta")

/home/yelf/.pyenv/torch2.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yelf/.pyenv/torch2.4/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [13]:
path = "akahana/indonesia-sentiment-roberta"
emotion = pipeline('text-classification', 
                     model=path,device=0)

kalimat = "Anak anjing lu pada pantek"
preds = emotion(kalimat)
preds

[{'label': 'NEGATIF', 'score': 0.9933290481567383}]